In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# quality of the image
mpl.rcParams["figure.dpi"] = 150


In [13]:
# data
client_tr = pd.read_csv("data/client.train.csv", parse_dates=["birthdate"])
client_te = pd.read_csv("data/client.test.csv", parse_dates=["birthdate"])
client_ch = pd.read_csv("data/client.challenge.csv", parse_dates=["birthdate"])
orders = pd.read_csv("data/orders.csv", parse_dates=["datetime"])
product = pd.read_csv("data/product.csv")

product

,product_id,category_lv0,category_lv1,category_lv2,category_lv3,category_lv4
0,px01237,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:04846
1,px11510,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:01337
2,px03782,mk38,mk38:072,mk38:072:019,mk38:072:019:066,mk38:072:019:066:02321
3,px04092,mk38,mk38:072,mk38:072:019,mk38:072:019:054,mk38:072:019:054:06035
4,px07429,mk38,mk38:072,mk38:072:078,mk38:072:078:182,mk38:072:078:182:07218
...,...,...,...,...,...,...
12654,px11965,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:00392
12655,px05192,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:02028
12656,px05391,mk38,mk38:107,mk38:107:357,mk38:107:357:182,mk38:107:357:182:08006
12657,px00861,mk35,mk35:072,mk35:072:177,mk35:072:177:182,mk35:072:177:182:03018


In [27]:
client_tr["client_id"].unique().size

23292

In [28]:
client_tr

,client_id,birthdate,state,gender,is_churn
0,cx05072,1972-03-22,st25,cg2,False
1,cx03836,1966-08-14,st15,cg1,False
2,cx08472,1954-08-07,st15,cg1,False
3,cx12946,1955-11-16,st15,cg1,False
4,cx19045,1958-10-25,st15,cg2,False
...,...,...,...,...,...
23287,cx05312,1981-08-29,st18,cg1,False
23288,cx02434,1997-03-15,st04,cg2,False
23289,cx16024,1950-10-09,st11,cg1,False
23290,cx11285,1976-07-03,st11,cg2,False


In [15]:
orders

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source
0,ox008433,cx05072,px04611,2019-10-07 15:20:00,1.0,49.90,pm05,NaN,NaN,st07,dv9,sc08
1,ox070389,cx05072,px12305,2019-10-11 17:42:00,1.0,69.90,pm05,NaN,NaN,st07,dv9,sc08
2,ox061137,cx06840,px07033,2019-01-19 13:09:00,1.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
3,ox061137,cx06840,px05992,2019-01-19 13:09:00,3.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
4,ox038612,cx06840,px10040,2019-04-18 14:36:00,1.0,39.00,pm05,NaN,NaN,st07,dv9,sc08
...,...,...,...,...,...,...,...,...,...,...,...,...
168467,ox081422,cx15796,px08541,2019-06-25 11:50:00,1.0,246.50,pm14,10.0,21.0,st01,dv5,sc11
168468,ox081422,cx15796,px05166,2019-06-25 11:50:00,1.0,652.50,pm14,10.0,21.0,st01,dv5,sc11
168469,ox128219,cx15796,px12043,2019-09-06 11:45:00,2.0,15.00,pm05,NaN,NaN,st07,dv9,sc08
168470,ox126589,cx15796,px12043,2019-09-07 04:39:00,1.0,15.00,pm10,1.0,0.0,st01,dv3,sc12


In [23]:
# join orders + product
orders_product_df = pd.merge(
    left=orders,
    right=product,
    on="product_id",
    how="left"
)

In [24]:
orders_product_df.columns

Index(['order_id', 'client_id', 'product_id', 'datetime', 'units', 'price',
       'payment_method', 'installments', 'delivery_days', 'delivery_state',
       'device', 'source', 'category_lv0', 'category_lv1', 'category_lv2',
       'category_lv3', 'category_lv4'],
      dtype='object')

In [25]:
orders_product_df[orders_product_df["client_id"] == "cx05072"]

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source,category_lv0,category_lv1,category_lv2,category_lv3,category_lv4
0,ox008433,cx05072,px04611,2019-10-07 15:20:00,1.0,49.9,pm05,NaN,NaN,st07,dv9,sc08,mk04,mk04:128,mk04:128:103,mk04:128:103:103,mk04:128:103:103:03294
1,ox070389,cx05072,px12305,2019-10-11 17:42:00,1.0,69.9,pm05,NaN,NaN,st07,dv9,sc08,mk04,mk04:292,mk04:292:246,mk04:292:246:103,mk04:292:246:103:04338
